### Separar para cada fichero de entrada de catastro (.cat), los ficheros de salida de interes (tipo 11, tipo 15 y tipo 14). Los ficheros de entrada contienen todos los tipos mezclados, identificando cada uno por el comienzo de línea. Cada tipo de tabla tiene una especificación de ancho fijo

La descripción de los ficheros .cat se encuentra en http://www.catastro.minhap.es/documentos/formatos_intercambio/catastro_fin_cat_2006.pdf

In [ ]:
### imports y definición de directorios
import pandas as pd
import os
import numpy as np

processed_dir = '../../data/raw/CAT/procesados/'
pending_dir = '../../data/raw/CAT/pendientes/'
schema_dir = '../../data/clean/aux_files/'
data_dir = '../../data/clean/CAT/'

In [ ]:
pending_files = [f for f in os.listdir(pending_dir) if os.path.isfile(os.path.join(pending_dir, f))]

In [ ]:
schema_list = [f for f in os.listdir(schema_dir) if os.path.isfile(os.path.join(schema_dir, f))]

In [ ]:
schema_list = ([x for x in schema_list if str(x).find('.csv') > -1])

In [ ]:
schema_list.sort()

In [ ]:
pending_files

In [ ]:
for f in pending_files:
    in_process_file = os.path.join(pending_dir, f)
    
    for schema in schema_list:
        schema_file = os.path.join(schema_dir, schema)
        schema_type = '^' + schema[1:3]
        out_file = schema_type.replace('^', 'T') + '_' + in_process_file.split("/")[-1].split(".")[0] + '.csv'
        out_file = os.path.join(data_dir, out_file)
        #falta añadir el cut con las columnas de interes para cada schema
        !zcat $in_process_file|grep -a $schema_type|in2csv -e latin1 -s $schema_file > $out_file
        !gzip $out_file
        
    !mv $in_process_file $processed_dir
        

### Base de información tipo 15, al que se añade informanción relevante de tipos 11, 13 y 14

tipo11: parcelas catastrales, coordenadas xy

tipo13: unidades constructivas en parcelas, año de la construcción

tipo14: construcciones dentro de las unidades constructivas: año antiguedad, año reforma si hay, superficie

tipo15: registro inmuebles. Tabla principal en la que poblar los datos relevantes de las otras tablas

In [ ]:
input_dir = data_dir
out_dir = '../../data/clean/ficheros_preparados/'
processed_dir = '../../data/clean/CAT/procesados/'

In [ ]:
pending_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

In [ ]:
pending_files = [f for f in pending_files if f[0:3] == 'T15']

In [ ]:
for f in pending_files:
    
    f15 = os.path.join(input_dir, f)    
    
    ## cargamos fichero tipo 15 (fincas)

    indata15 = pd.read_csv(f15, low_memory=False, dtype='str')
    cols_eliminar = indata15.columns[indata15.columns.str.contains('blanc')]
    cols_eliminar = cols_eliminar.union(['tipo_reg', 'ctrl1', 'ctrl2', 'cmun_origen', 'cod_zona_concent', 'cod_poligono', 'cod_parcela', 'cod_paraje', 'nombre_paraje', 'num_orden_escrit_hori'])
    indata15 = indata15.drop(cols_eliminar, axis = 1)

    ## cargamos fichero tipo 14, para obtener m2 y años de construcción y reforma
    ## t15 y t14 se cruzan por (parcela, num_cargo) en t15 y (parcela, num_orden_BI) en t14. PUeden existir
    ## varios registros en t14 asociados a uno de 15. P. ej. viviendas con garajes. De aquí se puede marcar
    ## que viviendas tienen garaje incluido para luego estimar el coste

    f14 = f15.replace('T15', 'T14')
    indata14 = pd.read_csv(f14, low_memory=False, dtype='str')
    ## reducimos a sólo columnas de cruce y datos de interes
    cols_interes = ['parcela_cat', 'num_orden_BI', 'tipo_reforma', 'anio_ref', 'anio_antiguedad', 'local_interior', 'm2_total', 'm2_porches_terr', 'm2_imputables_loc', 'tipologia_constr', 'planta']
    indata14 = indata14.reindex(columns=cols_interes)

    ## muestra para acelerar desarrollo, comentar en producción
    #indata15 = indata15[0:200000]
    #indata14 = indata14[0:200000]

    ## dejamos en t14 sólo registros relacionados con t15 que sean viviendas, el resto no es de interés. Mejora
    ## rendimiento de groupby/transform posteriores
    indata15['parcela_cat'].fillna('No info', inplace = True)
    indata15['num_cargo'].fillna('No info', inplace = True)
    duplas_viv = indata15[indata15['clave_grupo_BI'] == 'V'].groupby(['parcela_cat', 'num_cargo'])
    duplas_viv = duplas_viv.size().reset_index()
    duplas_viv = duplas_viv.drop(0, axis = 1)
    indata14 = pd.merge(indata14, duplas_viv, left_on=['parcela_cat','num_orden_BI'], right_on=['parcela_cat', 'num_cargo'])


    ## seleccionamos las duplas de t14 con más de un registro y alguno en plantas negativas -> viviendas con garages
    indata14['parcela_cat'].fillna('No info', inplace = True)
    indata14['num_orden_BI'].fillna('No info', inplace = True)
    ## la línea de abajo es extremadamente lenta, preguntar pq. Sustituyo por group y merge
    #indata14['registros'] = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].transform(lambda x: len(x))
    grupo_comp = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].size().reset_index()
    indata14 = pd.merge(indata14, grupo_comp, how='left', left_on=['parcela_cat', 'num_orden_BI'], right_on=['parcela_cat', 'num_orden_BI'])
    indata14.rename(columns={0:'registros'}, inplace=True)
    indata14['planta'].fillna('No info', inplace = True)
    indata14['posible_garage'] = indata14.apply(lambda x: 1 if ((x['registros'] > 1) & (x['planta'][0] == '-')) else 0, axis=1)
    duplas14_garage = indata14[indata14['posible_garage'] == 1].groupby(['parcela_cat', 'num_orden_BI']).size().reset_index()
    duplas14_garage.drop(0, axis = 1, inplace=True)
    duplas14_garage['garage']=1

    ## marcamos en t15 las viviendas con garage
    indata15 = pd.merge(indata15, duplas14_garage, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
    indata15['garage'].fillna(0, inplace = True)
    indata15.drop('num_orden_BI', axis = 1, inplace = True)
    indata15['garage'] = indata15['garage'].map(int)

    ## dejamos un unico valor de año, el maximo de la reforma y de antiguedad

    years = ['anio_ref', 'anio_antiguedad']
    indata14[years] = indata14[years].apply(lambda x: x.fillna(0))
    indata14[years] = indata14[years].applymap(int)
    indata14['aniot14'] = np.maximum(indata14['anio_ref'], indata14['anio_antiguedad'])
    duplas14_maxyear = indata14.groupby(['parcela_cat', 'num_orden_BI'])['aniot14'].max().reset_index()
    indata15 = pd.merge(indata15, duplas14_maxyear, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
    years = ['antiguedad_BI', 'aniot14']
    indata15[years] = indata15[years].apply(lambda x: x.fillna(0))
    indata15[years] = indata15[years].applymap(int)
    indata15['anio_mejor'] = np.maximum(indata15['antiguedad_BI'], indata15['aniot14'])

    ## poblar con las coordenadas xy desde tipo 11 parcelas

    f11 = f15.replace('T15', 'T11')
    indata11 = pd.read_csv(f11, low_memory=False, dtype='str')
    ## Solo necesitamos las coordenadas y la proyección de esta tabla
    ## convertimos las coordenadas a float, los 2 ultimos digitos son el decimal
    cols_coor = indata11.columns[indata11.columns.str.contains('coor')]
    indata11[cols_coor] = indata11[cols_coor].apply(lambda x: x.fillna(0))
    indata11[cols_coor] = indata11[cols_coor].applymap(lambda x: int(x)/100.0)
    ## quitar 'EPSG:' de huso_geo
    indata11['huso_geo'].fillna('', inplace = True)
    indata11['huso_geo'] = indata11['huso_geo'].apply(lambda x: x[5:])

    indata15 = pd.merge(indata15, indata11[['parcela_cat', 'x_coor', 'y_coor', 'huso_geo']], how='left', left_on='parcela_cat', right_on='parcela_cat')

    ## grabamos el fichero de resultado 
    fbienesinmuebles = f.replace('T15', 'BI')
    fbienesinmuebles = f.replace('.gz', '')
    fbienesinmuebles = os.path.join(out_dir, fbienesinmuebles)

    indata15.to_csv(fbienesinmuebles, index=False)
    !gzip -f $fbienesinmuebles

    ## movemos los ficheros originales al directorio de procesados
    !mv $f11 $processed_dir
    !mv $f13 $processed_dir
    !mv $f15 $processed_dir



In [ ]:

indata15.rename(columns={'garage':'vivienda_con_garage'}, inplace=True)

## buscamos el año de antiguedad y de reforma si la ha habido. Sólo nos interesan viviendas


## cargamos los ficheros tipo11 (parcelas)
f11 = f.replace('T15', 'T11')
in_data_11 = pd.read_csv(f11, low_memory=False, dtype='str')

## eliminamos las columnas en blanco
cols_eliminar = in_data_11.columns[in_data_11.columns.str.contains('blanc')]
## añadimos ademas otras columnas sin interes
cols_eliminar = cols_eliminar.union(['tipo_reg', 'cmun_origen', 'cod_zona_concent', 'cod_poligono', 'cod_parcela', 'cod_paraje', 'nombre_paraje', 'refcat_BICE', 'denom_BICE'])
in_data_11 = in_data_11.drop(cols_eliminar, axis=1)
## convertimos todos los valores de m2 a formato número
in_data_11[cols_metros] = in_data_11[cols_metros].applymap(int)



In [ ]:
f = os.path.join(input_dir, 'T15_28_900_U_2016-01-23.csv.gz')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
indata15[(indata15['parcela_cat']=='8776301VK4787F') & (indata15['num_cargo']=='0114') ]

In [1]:
### imports y definición de directorios
import pandas as pd
import os
import numpy as np

input_dir = '../../data/clean/CAT/'
out_dir = '../../data/clean/ficheros_preparados/'
processed_dir = '../../data/clean/CAT/procesados/'

f = 'T15_28_900_U_2016-01-23.csv.gz'
f15 = os.path.join(input_dir, f)


In [ ]:
f15

In [2]:
## cargamos fichero tipo 15 (fincas)

indata15 = pd.read_csv(f15, low_memory=False, dtype='str')
cols_eliminar = indata15.columns[indata15.columns.str.contains('blanc')]
cols_eliminar = cols_eliminar.union(['tipo_reg', 'ctrl1', 'ctrl2', 'cmun_origen', 'cod_zona_concent', 'cod_poligono', 'cod_parcela', 'cod_paraje', 'nombre_paraje', 'num_orden_escrit_hori'])
indata15 = indata15.drop(cols_eliminar, axis = 1)

## cargamos fichero tipo 14, para obtener m2 y años de construcción y reforma
## t15 y t14 se cruzan por (parcela, num_cargo) en t15 y (parcela, num_orden_BI) en t14. PUeden existir
## varios registros en t14 asociados a uno de 15. P. ej. viviendas con garajes. De aquí se puede marcar
## que viviendas tienen garaje incluido para luego estimar el coste

f14 = f15.replace('T15', 'T14')
indata14 = pd.read_csv(f14, low_memory=False, dtype='str')
## reducimos a sólo columnas de cruce y datos de interes
cols_interes = ['parcela_cat', 'num_orden_BI', 'tipo_reforma', 'anio_ref', 'anio_antiguedad', 'local_interior', 'm2_total', 'm2_porches_terr', 'm2_imputables_loc', 'tipologia_constr', 'planta']
indata14 = indata14.reindex(columns=cols_interes)

## muestra para acelerar desarrollo, comentar en producción
#indata15 = indata15[0:200000]
#indata14 = indata14[0:200000]

## dejamos en t14 sólo registros relacionados con t15 que sean viviendas, el resto no es de interés. Mejora
## rendimiento de groupby/transform posteriores
indata15['parcela_cat'].fillna('No info', inplace = True)
indata15['num_cargo'].fillna('No info', inplace = True)
duplas_viv = indata15[indata15['clave_grupo_BI'] == 'V'].groupby(['parcela_cat', 'num_cargo'])
duplas_viv = duplas_viv.size().reset_index()
duplas_viv = duplas_viv.drop(0, axis = 1)
indata14 = pd.merge(indata14, duplas_viv, left_on=['parcela_cat','num_orden_BI'], right_on=['parcela_cat', 'num_cargo'])


## seleccionamos las duplas de t14 con más de un registro y alguno en plantas negativas -> viviendas con garages
indata14['parcela_cat'].fillna('No info', inplace = True)
indata14['num_orden_BI'].fillna('No info', inplace = True)
## la línea de abajo es extremadamente lenta, preguntar pq. Sustituyo por group y merge
#indata14['registros'] = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].transform(lambda x: len(x))
grupo_comp = indata14.groupby(['parcela_cat', 'num_orden_BI'])[['parcela_cat']].size().reset_index()
indata14 = pd.merge(indata14, grupo_comp, how='left', left_on=['parcela_cat', 'num_orden_BI'], right_on=['parcela_cat', 'num_orden_BI'])
indata14.rename(columns={0:'registros'}, inplace=True)
indata14['planta'].fillna('No info', inplace = True)
indata14['posible_garage'] = indata14.apply(lambda x: 1 if ((x['registros'] > 1) & (x['planta'][0] == '-')) else 0, axis=1)
duplas14_garage = indata14[indata14['posible_garage'] == 1].groupby(['parcela_cat', 'num_orden_BI']).size().reset_index()
duplas14_garage.drop(0, axis = 1, inplace=True)
duplas14_garage['garage']=1

## marcamos en t15 las viviendas con garage
indata15 = pd.merge(indata15, duplas14_garage, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
indata15['garage'].fillna(0, inplace = True)
indata15.drop('num_orden_BI', axis = 1, inplace = True)
indata15['garage'] = indata15['garage'].map(int)

## dejamos un unico valor de año, el maximo de la reforma y de antiguedad

years = ['anio_ref', 'anio_antiguedad']
indata14[years] = indata14[years].apply(lambda x: x.fillna(0))
indata14[years] = indata14[years].applymap(int)
indata14['aniot14'] = np.maximum(indata14['anio_ref'], indata14['anio_antiguedad'])
duplas14_maxyear = indata14.groupby(['parcela_cat', 'num_orden_BI'])['aniot14'].max().reset_index()
indata15 = pd.merge(indata15, duplas14_maxyear, how='left', left_on=['parcela_cat', 'num_cargo'], right_on=['parcela_cat','num_orden_BI'])
years = ['antiguedad_BI', 'aniot14']
indata15[years] = indata15[years].apply(lambda x: x.fillna(0))
indata15[years] = indata15[years].applymap(int)
indata15['anio_mejor'] = np.maximum(indata15['antiguedad_BI'], indata15['aniot14'])

## poblar con las coordenadas xy desde tipo 11 parcelas

f11 = f15.replace('T15', 'T11')
indata11 = pd.read_csv(f11, low_memory=False, dtype='str')
## Solo necesitamos las coordenadas y la proyección de esta tabla
## convertimos las coordenadas a float, los 2 ultimos digitos son el decimal
cols_coor = indata11.columns[indata11.columns.str.contains('coor')]
indata11[cols_coor] = indata11[cols_coor].apply(lambda x: x.fillna(0))
indata11[cols_coor] = indata11[cols_coor].applymap(lambda x: int(x)/100.0)
## quitar 'EPSG:' de huso_geo
indata11['huso_geo'].fillna('', inplace = True)
indata11['huso_geo'] = indata11['huso_geo'].apply(lambda x: x[5:])

indata15 = pd.merge(indata15, indata11[['parcela_cat', 'x_coor', 'y_coor', 'huso_geo']], how='left', left_on='parcela_cat', right_on='parcela_cat')



In [3]:
indata14.head()

,parcela_cat,num_orden_BI,tipo_reforma,anio_ref,anio_antiguedad,local_interior,m2_total,m2_porches_terr,m2_imputables_loc,tipologia_constr,planta,num_cargo,registros,posible_garage,aniot14
0,0148301VK4704G,0093,NaN,0,1984,N,0000063,0000053,0000000,01122,07,0093,1,0,1984
1,0148301VK4704G,0094,NaN,0,1984,N,0000045,0000049,0000000,01122,07,0094,1,0,1984
2,0148301VK4704G,0095,NaN,0,1984,N,0000053,0000035,0000000,01122,07,0095,1,0,1984
3,0148301VK4704G,0096,NaN,0,1984,N,0000065,0000056,0000000,01122,07,0096,1,0,1984
4,0148301VK4704G,0097,NaN,0,1984,N,0000061,0000051,0000000,01122,07,0097,1,0,1984


In [ ]:
indata15['garage'].value_counts()

In [4]:
indata15.head()


,cod_del_MEH,cod_muni_DGC,clase_BI,parcela_cat,num_cargo,num_fijo_BI,BI_ayunt,num_finca_registral,cprov,provincia,...,m2_BI,m2_solares_sin_div_hor,coef_finca,garage,num_orden_BI,aniot14,anio_mejor,x_coor,y_coor,huso_geo
0,28,900,UR,1965204VK4716F,0001,00257650,000000000255496,00000,28,MADRID,...,0000021526,0000002321,100000000,0,NaN,0,1975,441827.64,4476294.63,25830
1,28,900,UR,9437605VK3793G,0001,81140056,000000000063083,00000,28,MADRID,...,0000000000,0000001150,100000000,0,NaN,0,0,439356.70,4473478.15,25830
2,28,900,UR,0148301VK4704G,0001,00001774,000000000079248,2810802002614,28,MADRID,...,0000000023,0000000631,000070000,0,NaN,0,1984,439945.24,4474677.58,25830
3,28,900,UR,0148301VK4704G,0002,00001775,000000000079249,2810802002616,28,MADRID,...,0000000023,0000000631,000070000,0,NaN,0,1984,439945.24,4474677.58,25830
4,28,900,UR,0148301VK4704G,0003,00001776,000000000079250,2810802002618,28,MADRID,...,0000000023,0000000631,000070000,0,NaN,0,1984,439945.24,4474677.58,25830


In [ ]:
indata15['anio_mejor'].value_counts()

In [5]:
## grabamos el fichero de resultado 
fbienesinmuebles = f.replace('T15', 'BI')
fbienesinmuebles = fbienesinmuebles.replace('.gz', '')
fbienesinmuebles = os.path.join(out_dir, fbienesinmuebles)

In [8]:
fbienesinmuebles

'../../data/clean/ficheros_preparados/BI_28_900_U_2016-01-23.csv'

In [ ]:
f

In [6]:
indata15.to_csv(fbienesinmuebles, index=False)
!gzip -f $fbienesinmuebles

In [7]:
## movemos los ficheros originales al directorio de procesados
!mv $f11 $processed_dir
!mv $f14 $processed_dir
!mv $f15 $processed_dir

In [10]:
indata11.to_csv('../../data/clean/ficheros_preparados/parcelas.csv', index=False)